In [1]:
# Regular EDA(exploratory data analysis) and plotting libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
%matplotlib inline

from tqdm import tqdm
import string
import statistics
import re
import joblib
import random
import statistics
from collections import Counter
from scipy import stats
import os

# import seaborn as sns
from mpl_toolkits.mplot3d import Axes3D
from sklearn.metrics.pairwise import cosine_similarity
import zipfile
from wordcloud import WordCloud
from matplotlib.lines import Line2D

In [2]:
# Import the Data
mir_link_id = pd.read_csv('/kaggle/input/media-study/mir_afsar_ali_yt_links - Sheet1.csv')


def extract_video_id(url):
    # Regular expression pattern to match YouTube video IDs
    pattern = r'(?:youtube\.com\/(?:[^\/\n\s]+\/\S+\/|(?:v|e(?:mbed)?)\/|\S*?[?&]v=)|youtu\.be\/)([a-zA-Z0-9_-]{11})'

    # Search for the video ID in the URL
    match = re.search(pattern, url)

    if match:
        return match.group(1)  # Return the matched video ID
    else:
        return None  # Return None if no match found

# Example URLs
urls = mir_link_id['yt_link']

video_ids = [extract_video_id(url) for url in urls]
mir_link_id['video_id'] = video_ids

print(mir_link_id.category.value_counts())


mirer_thek_vids = mir_link_id[mir_link_id['category'] == 'mirerthek']['video_id'].tolist()
mir_soho_vids = mir_link_id[mir_link_id['category'] == 'mir soho']['video_id'].tolist()
mir_chara_vids = mir_link_id[mir_link_id['category'] == 'mir chara']['video_id'].tolist()

print(f'{len(mirer_thek_vids)} {len(mir_soho_vids)} {len(mir_chara_vids)}')

category
mir soho     304
mir chara    168
mirerthek     62
Name: count, dtype: int64
62 304 168


In [3]:
print(len(set(mir_soho_vids)))

202


In [2]:
dfc = pd.read_csv('/kaggle/input/media-study2-0/final_video_info/mir_chara_vif.csv')
dfs = pd.read_csv('/kaggle/input/media-study2-0/final_video_info/mir_soho_vif.csv')
dft = pd.read_csv('/kaggle/input/media-study2-0/final_video_info/mirer_thek_vif.csv')

print(f'{dfc.shape} {dfs.shape} {dft.shape}')

(246, 9) (180, 9) (70, 9)


## Mir Chara Sunday Suspense

In [3]:
mc_com = pd.read_parquet('/kaggle/input/media-study2-0/final_comment_data/mir_chara_comment.parquet')
mc_chat = pd.read_csv('/kaggle/input/media-study2-0/all_live_chat/mir_chara_live_chat.csv')

ms_com = pd.read_parquet('/kaggle/input/media-study2-0/final_comment_data/mir_soho_comment.parquet')
ms_chat = pd.read_csv('/kaggle/input/media-study2-0/all_live_chat/mir_shoho_live_chat.csv')

mt_com = pd.read_csv('/kaggle/input/media-study2-0/final_comment_data/mirer_thek_comment.csv')
mt_chat = pd.read_csv('/kaggle/input/media-study2-0/all_live_chat/mirer_thek_live_chat.csv')

print(f'Length of Mir Chara Comments is {mc_com.shape}')
print(f'Length of Mir Chara live chat is {mc_chat.shape}')

print('Total Videos Scraped', mc_com['video_id'].nunique())

mc_com.head(3)

Length of Mir Chara Comments is (201633, 6)
Length of Mir Chara live chat is (343125, 4)
Total Videos Scraped 245


,author,published_date,published_time,like_count,text,video_id
0,@skmdimran852,2024-05-21,03:45:59Z,0,❤❤❤,SgqeN_FUprg
1,@tanvirhaquekhan6061,2024-05-20,14:25:10Z,0,সত্যজিত রায়ের মতো এমন দূরদর্শী মানুষ হাজার বছর...,SgqeN_FUprg
2,@MohammadArman-ob1id,2024-05-20,12:12:24Z,0,Mitryur porer abostha toh bollen na keno ami j...,SgqeN_FUprg


In [4]:
mc_chat.head(3)

,datetime,author,message,vid_id
0,2024-05-05 07:28:34,Paglamo Sudupaglamo,joy east bengal :red_square::yellow_square::re...,SgqeN_FUprg
1,2024-05-05 07:28:34,Soumen banarjee🙂,hi,SgqeN_FUprg
2,2024-05-05 07:28:37,Paglamo Sudupaglamo,joy east bengal :red_square::yellow_square::re...,SgqeN_FUprg


In [5]:
mc_com.drop(columns = ['published_time', 'video_id'], inplace=True)
ms_com.drop(columns = ['published_time', 'video_id'], inplace=True)
mt_com.drop(columns = ['published_time', 'video_id'], inplace=True)

mc_com.head(3)

,author,published_date,like_count,text
0,@skmdimran852,2024-05-21,0,❤❤❤
1,@tanvirhaquekhan6061,2024-05-20,0,সত্যজিত রায়ের মতো এমন দূরদর্শী মানুষ হাজার বছর...
2,@MohammadArman-ob1id,2024-05-20,0,Mitryur porer abostha toh bollen na keno ami j...


In [6]:
# Assuming df is your DataFrame
mc_com['text'] = mc_com['text'].astype(str)
ms_com['text'] = ms_com['text'].astype(str)
mt_com['text'] = mt_com['text'].astype(str)
mc_chat['message'] = mc_chat['message'].astype(str)

In [7]:
import langid

def detect_language(text):
    x = langid.classify(text)[0]
    if  x == 'bn':
        return 'bangla'
    elif x == 'en':
        return 'english'
    else:
        return 'banglish'


In [8]:
print(mt_com['text'][0])
detect_language(mt_com['text'][0])

Thanks Mr.mir


'english'

In [9]:
import langid
def clean_emoji(text):
    cleaned_text = re.sub('[^a-zA-z0-9\u0980-\u09FF,./!?$%&"\'\\-_;:।]'," ",text)
    return cleaned_text

tqdm.pandas()
ms_com['ctext'] = ms_com.text.progress_apply(clean_emoji)
mc_com['ctext'] = mc_com.text.progress_apply(clean_emoji)
mt_com['ctext'] = mt_com.text.progress_apply(clean_emoji)

100%|██████████| 99687/99687 [00:01<00:00, 88818.98it/s]


In [11]:
def detect_whitespace(text):
    return re.match(r'^\s*$', text) is not None

ms_com['language'] = ''
mc_com['language'] = ''
mt_com['language'] = ''


mt_com.head()

,author,published_date,like_count,text,public,ctext,language
0,@anwarulkabir6319,2024-05-16,0,Thanks Mr.mir,True,Thanks Mr.mir,
1,@JayitaKundu-fy3hp,2024-05-11,0,বিভূতিভূষণ বন্দ্যোপাধ্যায় এর আদৰ্শ হিন্দু হোটে...,True,বিভূতিভূষণ বন্দ্যোপাধ্যায় এর আদৰ্শ হিন্দু হোটে...,
2,@subi4098,2024-05-04,0,.vison boro fan tomar ami..mirchi te tomar gol...,True,.vison boro fan tomar ami..mirchi te tomar gol...,
3,@samhotibanerjee95,2024-05-03,0,এতো স্ট্রোরি টেলার আছে yt channel এ কিন্তু মীর...,True,এতো স্ট্রোরি টেলার আছে yt channel এ কিন্তু মীর...,
4,@malabikasengupta9803,2024-04-29,0,Khub bhalo laglo Mir.\nMone hochilo golper mod...,True,Khub bhalo laglo Mir. Mone hochilo golper modh...,


In [12]:
mc_com.isna().sum()

author            0
published_date    0
like_count        0
text              0
ctext             0
language          0
dtype: int64

In [13]:
for ind in tqdm(mt_com.index):
    if detect_whitespace(mt_com['ctext'][ind]) == False:
        try:
            lang=langid.classify(mt_com['ctext'][ind])
        except:
            lang[0]='no'
        if lang[0] == "en":
            mt_com.loc[ind,'language'] = "English"
        elif lang[0] == "bn":
            mt_com.loc[ind,'language'] = "Bengali"
        elif lang[0] == "no":
            mt_com.loc[ind,'language'] = "Unidentified"
        else:
            mt_com.loc[ind,'language'] = "Banglish"
    elif detect_whitespace(mt_com['ctext'][ind]) == True:
        mt_com.loc[ind,'language'] = "Emoji"

100%|██████████| 99687/99687 [05:15<00:00, 316.39it/s]


In [16]:
mt_com.head(3)

,author,published_date,like_count,text,public,ctext,language
0,@anwarulkabir6319,2024-05-16,0,Thanks Mr.mir,True,Thanks Mr.mir,English
1,@JayitaKundu-fy3hp,2024-05-11,0,বিভূতিভূষণ বন্দ্যোপাধ্যায় এর আদৰ্শ হিন্দু হোটে...,True,বিভূতিভূষণ বন্দ্যোপাধ্যায় এর আদৰ্শ হিন্দু হোটে...,Bengali
2,@subi4098,2024-05-04,0,.vison boro fan tomar ami..mirchi te tomar gol...,True,.vison boro fan tomar ami..mirchi te tomar gol...,Banglish


In [17]:
mt_com.language.value_counts()

language
Bengali         41638
Banglish        26659
English         23875
Emoji            6659
Unidentified      856
Name: count, dtype: int64

In [18]:
filt = mt_com[mt_com['language']=='Emoji']
filt

,author,published_date,like_count,text,public,ctext,language
52,@anupanda6922,2023-12-03,0,🙏❤,True,,Emoji
93,@pulakdebnath4507,2023-09-03,0,❤,True,,Emoji
166,@avijit9857,2023-06-02,0,💖💖💖,True,,Emoji
172,@soumikmondal1741,2023-05-23,0,❤❤❤,True,,Emoji
182,@swagatabanerjee4217,2023-05-18,0,❤,True,,Emoji
...,...,...,...,...,...,...,...
99609,@realmir,2024-05-11,1,❤❤❤❤❤❤❤❤❤,True,,Emoji
99631,@Siyamislam420,2024-05-11,0,❤❤❤❤,True,,Emoji
99640,@SutapaTamili-bb3cw,2024-05-11,0,❤❤❤❤❤❤,True,,Emoji
99645,@mousumighosh5676,2024-05-11,0,❤❤❤❤❤❤❤❤❤❤❤,True,,Emoji


In [19]:
filt.isna().sum()

author            0
published_date    0
like_count        0
text              0
public            0
ctext             0
language          0
dtype: int64

In [26]:
mt_com.dropna(inplace=True)
mt_com.isna().sum()

author            0
published_date    0
like_count        0
text              0
public            0
ctext             0
language          0
dtype: int64

In [27]:
mt_com.to_parquet('/kaggle/working/mirer_thek_comment.parquet', index=False)

In [30]:
for ind in tqdm(mc_com.index):
    if detect_whitespace(mc_com['ctext'][ind]) == False:
        try:
            lang=langid.classify(mc_com['ctext'][ind])
        except:
            lang[0]='no'
        if lang[0] == "en":
            mc_com.loc[ind,'language'] = "English"
        elif lang[0] == "bn":
            mc_com.loc[ind,'language'] = "Bengali"
        elif lang[0] == "no":
            mc_com.loc[ind,'language'] = "Unidentified"
        else:
            mc_com.loc[ind,'language'] = "Banglish"
    elif detect_whitespace(mc_com['ctext'][ind]) == True:
        mc_com.loc[ind,'language'] = "Emoji"

100%|██████████| 201633/201633 [15:28<00:00, 217.14it/s]


In [31]:
mc_com.head(3)

,author,published_date,like_count,text,ctext,language
0,@koushikdas7808,2022-07-31,6850,মীর কে লক্ষ লক্ষ মানুষ শুনতে ভালোবাসে। তাই আমি...,মীর কে লক্ষ লক্ষ মানুষ শুনতে ভালোবাসে। তাই আমি...,Bengali
1,@adityapaul2361,2022-11-20,5144,আমার মতো মীর দা ও তারানাথ তান্ত্রিককে কে কে চা...,আমার মতো মীর দা ও তারানাথ তান্ত্রিককে কে কে চা...,Bengali
2,@Jeetbose15,2022-10-08,4880,এক পুরনো শ্রোতা এবং Sunday Suspense এর শুভাকাঙ...,এক পুরনো শ্রোতা এবং Sunday Suspense এর শুভাকাঙ...,Bengali


In [32]:
mc_com.language.value_counts()

language
Banglish        65839
Bengali         65227
English         59780
Emoji            8887
Unidentified     1900
Name: count, dtype: int64

In [33]:
# mt_com.dropna(inplace=True)
mc_com.isna().sum()

author            0
published_date    0
like_count        0
text              0
ctext             0
language          0
dtype: int64

In [34]:
mc_com.to_parquet('/kaggle/working/mir_chara_comment.parquet', index=False)

In [ ]:
for ind in tqdm(mc_com.index):
    if detect_whitespace(mc_com['ctext'][ind]) == False:
        try:
            lang=langid.classify(mc_com['ctext'][ind])
        except:
            lang[0]='no'
        if lang[0] == "en":
            mc_com.loc[ind,'language'] = "English"
        elif lang[0] == "bn":
            mc_com.loc[ind,'language'] = "Bengali"
        elif lang[0] == "no":
            mc_com.loc[ind,'language'] = "Unidentified"
        else:
            mc_com.loc[ind,'language'] = "Banglish"
    elif detect_whitespace(mc_com['ctext'][ind]) == True:
        mc_com.loc[ind,'language'] = "Emoji"

In [28]:
print(mc_com.shape, ms_com.shape, mt_com.shape)

(201633, 6) (445398, 6) (99686, 7)


In [21]:
mt_com['text'][0]

'Thanks Mr.mir'

In [35]:
mc_com = mc_com.sort_values(by='like_count', ascending=False).reset_index(drop=True)
mc_com.head(5)

,author,published_date,like_count,text,ctext,language
0,@koushikdas7808,2022-07-31,6850,মীর কে লক্ষ লক্ষ মানুষ শুনতে ভালোবাসে। তাই আমি...,মীর কে লক্ষ লক্ষ মানুষ শুনতে ভালোবাসে। তাই আমি...,Bengali
1,@adityapaul2361,2022-11-20,5144,আমার মতো মীর দা ও তারানাথ তান্ত্রিককে কে কে চা...,আমার মতো মীর দা ও তারানাথ তান্ত্রিককে কে কে চা...,Bengali
2,@Jeetbose15,2022-10-08,4880,এক পুরনো শ্রোতা এবং Sunday Suspense এর শুভাকাঙ...,এক পুরনো শ্রোতা এবং Sunday Suspense এর শুভাকাঙ...,Bengali
3,@SarthakJasu,2023-03-05,4130,Mir Da সেরা ছিল but Deep দা তুমি যে এইভাবে আমা...,Mir Da সেরা ছিল but Deep দা তুমি যে এইভাবে আমা...,Bengali
4,@surajitbarman8525,2023-01-07,3562,কয়েকদিন পর থেকে সেমিস্টার এর পরীক্ষা শুরু হবে...,কয়েকদিন পর থেকে সেমিস্টার এর পরীক্ষা শুরু হবে...,Bengali


In [36]:
ms_com = ms_com.sort_values(by='like_count', ascending=False).reset_index(drop=True)
ms_com.head(5)

,author,published_date,like_count,text,ctext,language
0,@wolfhowling220,2021-12-19,5373,"Sunday suspense er শ্রোতাদের মধ্যে কে কে ""The ...","Sunday suspense er শ্রোতাদের মধ্যে কে কে ""The ...",Bengali
1,@ksarkar1338,2021-12-03,4929,বেঁচে থাকতে হয়তো বিশ্ব সাহিত্যের যেসব গল্প কো...,বেঁচে থাকতে হয়তো বিশ্ব সাহিত্যের যেসব গল্প কো...,Bengali
2,@dcsalim4384,2021-12-12,4830,এ আমার পরম সৌভাগ্য যে একজন বাঙালি হয়ে জন্মেছি ...,এ আমার পরম সৌভাগ্য যে একজন বাঙালি হয়ে জন্মেছি ...,Bengali
3,@AmioPariGuru,2022-01-22,4551,কে কে মনে করেন 'The count of Monte Christo' এই...,কে কে মনে করেন 'The count of Monte Christo' এই...,Bengali
4,@rishiupadhyay3956,2022-05-22,3953,"একটা ছোট্ট request, Alexender Dumas series যখন...","একটা ছোট্ট request, Alexender Dumas series যখন...",Bengali


In [37]:
mt_com = mt_com.sort_values(by='like_count', ascending=False).reset_index(drop=True)
mt_com.head(5)

,author,published_date,like_count,text,public,ctext,language
0,@SubashishChakraborty,2023-03-04,4209,"আজ ""ব্রহ্মপিশাচ"", কাল সানডে সাসপেন্সে"" জলপিশাচ...",True,"আজ ""ব্রহ্মপিশাচ"", কাল সানডে সাসপেন্সে"" জলপিশাচ...",Bengali
1,@sakilahamed5070,2023-09-16,2828,সোমক দা❤ কে গপ্পো মীরের ঠেকে নিয়মিত শুনতে চায...,True,সোমক দা কে গপ্পো মীরের ঠেকে নিয়মিত শুনতে চায...,Bengali
2,@gouravkayal4650,2023-08-26,2588,দাদার গলায় তারানাথ তান্ত্রিক চরিত্রটি শুনতে ক...,True,দাদার গলায় তারানাথ তান্ত্রিক চরিত্রটি শুনতে ক...,Bengali
3,@arunavakarmakar8147,2023-02-12,2453,কে কে চায় যে আবার মীরদার কন্ঠে তারানাথ তান্ত্র...,True,কে কে চায় যে আবার মীরদার কন্ঠে তারানাথ তান্ত্র...,Bengali
4,@gouravkayal4650,2023-08-12,1842,দাদার কণ্ঠে কে কে বিভূতিবাবুর লেখা 'আরণ্যক' উপ...,True,দাদার কণ্ঠে কে কে বিভূতিবাবুর লেখা 'আরণ্যক' উপ...,Bengali


In [53]:
def remove_extra_spaces(text):
    # Split the text by whitespace and then join it back with a single space
    cleaned_text = ' '.join(text.split())
    return cleaned_text

ms_com['ctext'] = ms_com['ctext'].apply(remove_extra_spaces)
mc_com['ctext'] = mc_com['ctext'].apply(remove_extra_spaces)
mt_com['ctext'] = mt_com['ctext'].apply(remove_extra_spaces)

In [56]:
# say threshold is 20k
threshold = 10_00

test_textms = ms_com[:threshold]['ctext'].tolist()
lang_ms = ms_com[:threshold]['language'].tolist()

print(test_textms[:1])
print(lang_ms[:1])

test_textmc = mc_com[:threshold]['ctext'].tolist()
lang_mc = mc_com[:threshold]['language'].tolist()

print(test_textmc[:1])
print(lang_mc[:1])


test_textmt = mt_com[:threshold]['ctext'].tolist()
lang_mt = mt_com[:threshold]['language'].tolist()

print(test_textmt[:1])
print(lang_mt[:1])


print(f'\n{len(test_textmc)}')

['Sunday suspense er শ্রোতাদের মধ্যে কে কে "The count of Monte Cristo" শুনেছ ?']
['Bengali']
['মীর কে লক্ষ লক্ষ মানুষ শুনতে ভালোবাসে। তাই আমি মনে করি শ্রোতাদের কথা মাথায় রেখে Sunday suspense টা তার চালিয়ে যাওয়া উচিত ছিল। যেরকম দীপ এখনো করে। মীরকে ছারা ব্যোমকেশ, তারানাথ চরিত্র গুলো ভাবা যায়না। আপনাদের কি মতামত?']
['Bengali']
['আজ "ব্রহ্মপিশাচ", কাল সানডে সাসপেন্সে" জলপিশাচ"। জমে যাবে পুরো। কিন্তু যতই বলো, তারানাথের চরিত্রটি মীরের কন্ঠ ছাড়া মানায় না।']
['Bengali']

1000


In [ ]:
# pip install git+https://github.com/csebuetnlp/normalizer

In [39]:
import torch
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)
device

cpu


device(type='cpu')

In [40]:
from transformers import pipeline
classifier = pipeline("text-classification",model='bhadresh-savani/bert-base-uncased-emotion', return_all_scores=False)

2024-06-10 10:19:21.345772: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-10 10:19:21.346009: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-10 10:19:21.540049: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


config.json:   0%|          | 0.00/935 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/285 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/transformers/pipelines/text_classification.py:104: UserWarning: `return_all_scores` is now deprecated,  if want a similar functionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(


In [41]:
prediction = classifier("I love using transformers. The best part is wide range of support and its easy to use", )
prediction[0]

{'label': 'joy', 'score': 0.9972521662712097}

In [44]:
ri = np.random.randint(0, 10_00)
print((test_textmt[ri]))
print(classifier(test_textmt[ri]))

# predict(test_textmt[ri], model)

গাঁয়ে কাঁটা দিয়ে উঠছে শুনে❤️❤️ চোখ দিয়ে জল এসে গেল এত সুন্দর করে উপস্থাপনা 🥺 বন্দেমাতরম ❤️❤️❤️
[{'label': 'joy', 'score': 0.8894343972206116}]


In [45]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer
from transformers import AutoModelForPreTraining, AutoTokenizer
# from normalizer import normalize 


# MODEL BANGLA
# check_point = "csebuetnlp/banglishbert"
modelcp = '/kaggle/input/trained-models/emo_model_dis'
checkpoint = "csebuetnlp/banglabert"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForSequenceClassification.from_pretrained(modelcp)

print(model.config)

tokenizer_config.json:   0%|          | 0.00/119 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/586 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/528k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

ElectraConfig {
  "_name_or_path": "/kaggle/input/trained-models/emo_model_dis",
  "architectures": [
    "ElectraForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "embedding_size": 768,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2,
    "LABEL_3": 3
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "electra",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "problem_type": "single_label_classification",
  "summary_activation": "gelu",
  "summary_last_dropout": 0.1,
  "summary_type": "first",
  "summary_use_proj": true,
  "torch_dtype": "float32",
  "transformers_version": "4.38.2

In [46]:
new_label_mapping = {
     0: "love",
     1: "sad",
     2: "anger",
     3: "fear"
}
model.config.id2label = new_label_mapping
model.config.label2id = {v: int(k) for k, v in new_label_mapping.items()}

In [47]:
input_text = 'অনেক ভালো গল্প।'
# input_text = 'খুব বাজে গল্প। অনেক খারাপ লাগছে শুনতে।'
# input_text = "very bad story."
print(tokenizer.tokenize(input_text))

['অনেক', 'ভালো', 'গল্প', '।']


In [48]:
model.config.id2label[2]

'anger'

In [49]:
input_text = 'অনেক ভালো গল্প।'
# input_text = "মীর ছাড়া sunday suspense শুনতে ভালো লাগে  না"
# input_text = "very bad story."
print(tokenizer.tokenize(input_text))


inputs = tokenizer(input_text, return_tensors="pt").to(device) # Tokenize and convert to tensors
inputs.pop("token_type_ids", None)
print(inputs)
model = model.to(device)
outputs = model(**inputs)  
logits = outputs.logits 
print(logits)
probabilities = torch.nn.functional.softmax(logits, dim=-1)
print(probabilities)

predicted_class_index = logits.argmax(-1).item()
predicted_class = model.config.id2label[predicted_class_index]
predicted_probability = probabilities[0][predicted_class_index].item()
print("Predicted sentiment:", predicted_class, "with probability:", predicted_probability)

['অনেক', 'ভালো', 'গল্প', '।']
{'input_ids': tensor([[   2, 1011, 1055, 1705,  205,    3]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1]])}
tensor([[ 7.6101, -2.0278, -3.2038, -2.7896]], grad_fn=<AddmmBackward0>)
tensor([[9.9988e-01, 6.5197e-05, 2.0116e-05, 3.0438e-05]],
       grad_fn=<SoftmaxBackward0>)
Predicted sentiment: love with probability: 0.9998842477798462


In [50]:
classifier(input_text)[0]['label']

'joy'

In [51]:
def predict(txt, model):
    '''
    Implements our awsome algorithm and gives final output
    '''
    inputs = tokenizer(txt, return_tensors="pt").to(device)
    model = model.to(device)
    outputs = model(**inputs)
    logits = outputs.logits
    probabilities = torch.nn.functional.softmax(logits, dim=-1)
    predicted_class_index = logits.argmax(-1).item()
    predicted_class = model.config.id2label[predicted_class_index]
    predicted_probability = probabilities[0][predicted_class_index].item()
    return [predicted_class, predicted_probability]

In [57]:
def final(text,lang, modelx):
    '''
    Implements our awsome algorithm and gives final output
    '''
    
#     print(x)
    if lang == 'Bengali':
        tokenized_text = tokenizer(text,
                                    return_tensors="pt").to(device)

        with torch.no_grad():
            modelx = modelx.to(device)
            pred = modelx(**tokenized_text)
            logits = pred.logits
            probas = torch.nn.functional.softmax(logits, dim=1).to('cpu')
            predicted_class = torch.argmax(probas, dim=1).item()
    #         print(predicted_class)
    #         x.append(probas.numpy().flatten())
        return model.config.id2label[predicted_class]
    
    else:
        return classifier(text)[0]['label']

In [60]:
for i in range(5):    
    ri = np.random.randint(0, len(mt_com))
    input_text = mc_com['ctext'][ri]
    lang = mc_com['language'][ri]
    print(input_text, lang)

    predicted_class = final(input_text, lang, model)
    print(predicted_class)
    print()

কর্নেল নীলাদ্রি সরকার জয়ন্ত হালদারমশাই The Three Musketeers of Syed Mustafa Siraj. কর্নেলের শান্ত,সৌম্য রূপে গোয়েন্দার থেকে প্রকৃতিবিদ ও পক্ষী-বিশারদ হিসেবেই বেশি মানায় Bengali
love

একদম ঠিক বলেছেন Bengali
love

soumyasaha6510 chorka কেনার টাকা আর তেল টা দিন ,দিয়ে দেবো Bengali
love

Now this compu is now our today's chatgpt. English
joy

পুরো গল্পটাই অসাধারণ লাগল Bengali
love



In [ ]:
ri = np.random.randint(0, len(test_textmc))
# ri = 8906
print(f'Index = {ri} \n{test_textmc[ri]}\n')
final(test_textmc[ri], model)

In [ ]:
preds1 = [final(i, model) for i in tqdm(test_textms)]

In [ ]:
preds2 = [final(i, model) for i in tqdm(test_textmc)]

In [ ]:
preds3 = [final(i, model) for i in tqdm(test_textmt)]

In [ ]:
result1 = pd.DataFrame({'comment':test_textms, 'sentiment':preds1})
result2 = pd.DataFrame({'comment':test_textmc, 'sentiment':preds2})
result3 = pd.DataFrame({'comment':test_textmt, 'sentimen':preds3})
result1

In [ ]:
for i, r in result1[:30].iterrows():
    print(r['sentiment'], r['comment'])
    print()

In [ ]:
print(result1.sentiment.value_counts().sort_index(),end='\n')
print(result2.sentiment.value_counts().sort_index(),end='\n')
print(result3.sentimen.value_counts().sort_index())

In [ ]:
result2.sentiment.value_counts().sort_index()

In [ ]:
spm = []
spm.append(result1.sentiment.value_counts().sort_index().tolist())
spm.append(result2.sentiment.value_counts().sort_index().tolist())
spm.append(result3.sentimen.value_counts().sort_index().tolist())
spm = np.array(spm)
spm

In [ ]:
sums = np.sum(spm, axis=1)
# Normalize each inner array to percentages
spm2 = (spm.T / sums).T * 100
print(spm2)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Define custom colors for each sentiment type
colors = ['#2E9BFF', '#987BE9', '#FF7C7C', '#58D68D', 'red', 'green', 'blue']

species = (
    "Mir Soho",  
    "Mir Chara", 
    "Mir Er Thek",  
)
weight_counts = {
    "Anger": spm2.T[0],
    "Fear": spm2.T[1],
    'Joy':spm2.T[2],
    "Love": spm2.T[3],
    "Sad": spm2.T[4],
    "Sadness": spm2.T[5],
    "Surprise":spm2.T[6]
}

# Calculate the number of sentiment types
num_sentiments = len(weight_counts)

# Adjust width based on the number of sentiment types
width = 0.4

fig, ax = plt.subplots(figsize=(8, 6))

# Initialize bottom values with zeros for stacking
bottom = np.zeros(len(species))

# Stack the bars for each sentiment type with custom colors
for i, (sentiment, count) in enumerate(weight_counts.items()):
    plt.bar(species, count, width, label=sentiment, bottom=bottom, color=colors[i])
    bottom += count  # Update bottom for stacking

# Add labels and title
plt.xlabel("Species")
plt.ylabel("Number of Occurrences")
plt.title("Distribution of Sentiment")

# Adjust legend location
plt.legend(loc="best")

plt.xticks(rotation=45)
plt.tight_layout()
plt.show()


## Ack
https://colab.research.google.com/drive/15h32YuSQ5YwVji8OynC9BvMaWGpsWyD2?usp=sharing#scrollTo=XGj0RdguKrDb
https://github.com/sagorbrur/bnlp/blob/main/docs/README.md#bengali-corpus-class